# tsam - Optimization Input Example

Usage of tsam to output relevant parameters for energy system optimization models.

This notebook shows how to access:
- Cluster weights (occurrence counts)
- Cluster assignments (period ordering)
- Typical period data

Import pandas and the relevant time series aggregation class

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import tsam
from tsam import ClusterConfig, ExtremeConfig

%matplotlib inline

### Input data 

Read in time series from testdata.csv with pandas

In [ ]:
raw = pd.read_csv("testdata.csv", index_col=0)

Transform the index to a datetime index

In [ ]:
raw.index = pd.to_datetime(raw.index)

Plot raw data

In [ ]:
axes = raw.plot(sharex=True, subplots=True)

### Aggregate the data

Aggregate to typical weeks, including days with minimum temperature and maximum load as extreme periods.

In [ ]:
result = tsam.aggregate(
    raw,
    n_periods=5,
    period_hours=24 * 7,  # Weekly periods
    cluster=ClusterConfig(method="hierarchical"),
    extremes=ExtremeConfig(
        method="new_cluster",
        min_timesteps=["T"],
        max_timesteps=["Load"],
    ),
)

Create the typical periods

In [ ]:
typPeriods = result.typical_periods
typPeriods.head()

### Show the resulting order  of aggregated periods

Calculates how the original index is represented by the old index

In [ ]:
# Access the internal aggregation object for advanced features
indexMatching = result._aggregation.indexMatching()
indexMatching.head()

Plot the appearance of the 5+2 aggregated periods in the original timeframe

In [ ]:
visDF = pd.DataFrame(0, index=indexMatching.index, columns=result.period_index)
for col in visDF.columns:
    visDF.loc[indexMatching["PeriodNum"] == col, col] = 1

In [ ]:
visDF.plot(kind="area", cmap="viridis", lw=0, ylim=[0, 1])

### Get input for potential energy system optimization

**i. cluster_weights** - The occurrence count of each typical period for weighting in the objective function.

Note: Period three is only partially evaluated since its appearance at the end of the year exceeds the original time series.

In [ ]:
result.cluster_weights

In [ ]:
ax = pd.Series(result.cluster_weights).plot(kind="bar")
ax.set(ylabel="Number of occurence", xlabel="Period index")

**ii. clusterPeriodDict**
<br>The dictionary which describes the aggregated time series by an index touple (period number, time step number) the resulting aggregated time series value for each time series, e.g. 'GHI' 

In [ ]:
# Access the internal aggregation object for dictionary-style access
agg = result._aggregation
agg.clusterPeriodDict["GHI"][(agg.clusterPeriodIdx[3], agg.stepIdx[12])]

Alternatively this is given as data frame

In [ ]:
result.typical_periods.head()

**iii. clusterOrder**
<br> The order of the typical periods to represent the original time series e.g. to model seasonal storage

In [ ]:
result.cluster_assignments